In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import pandas as pd

from io import BytesIO
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns

from feature_utils import normalize_histogram
from PIL import Image
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')


In [2]:
RED = 0
GREEN = 1
BLUE = 2

dataset_path = Path('./sign_data/')
if not dataset_path.joinpath('data').exists():
    dataset_path = Path('./')

In [3]:
# Read the parquet file, this takes a while. Leave it here
features_file = dataset_path.joinpath("data/train_features.parquet")
meta_file = dataset_path.joinpath("data/meta_full.parquet")


dft = pl.read_parquet(features_file, memory_map=True)
dfm = pl.read_parquet(meta_file, memory_map=True)
dft = dft.join(dfm, on="ClassId")
# del dfm

In [4]:
# To do: select 200 random images from each class
features = dft.filter(
    pl.int_range(0, pl.count()).shuffle().over("ClassId") < 200
)


In [5]:
# Test distribution:
features.groupby("ClassId").agg(pl.count())

ClassId,count
i64,u32
3,200
33,200
39,200
0,200
36,200
27,200
24,200
42,200
15,200


In [6]:
columns = ['ClassId', 'Hue_Hist', 'Saturation_Hist', 'Value_Hist', 'LBP_Hist', 'HOG_Features']
features = features[columns]


In [7]:
features.head(3)

ClassId,Hue_Hist,Saturation_Hist,Value_Hist,LBP_Hist,HOG_Features
i64,list[i64],list[i64],list[i64],list[i64],list[f32]
20,"[204, 1, … 0]","[19, 0, … 0]","[0, 0, … 0]","[150, 111, … 842]","[0.049189, 0.032555, … 0.093882]"
20,"[184, 4, … 5]","[42, 0, … 0]","[0, 0, … 0]","[121, 81, … 705]","[0.086153, 0.078746, … 0.0]"
20,"[181, 0, … 0]","[16, 0, … 0]","[0, 0, … 0]","[138, 119, … 987]","[0.124277, 0.043339, … 0.314378]"


In [8]:
def plot_models():
    plt.figure(figsize=(10,8))
    plt.suptitle("Models", fontsize=16, color="black", fontweight="bold")

    for i,m in enumerate(models):
        hist = pd.DataFrame(m[1].history)   
        hist = hist[np.isfinite(hist).all(1)]

        ax = plt.subplot(3, 2, i+1)

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.plot(range(len(hist)), hist['loss'], marker='.', color='black')
        plt.plot(range(len(hist)), hist['val_loss'], marker='.', color='red')
        plt.legend(['train loss', 'validation loss'])
        plt.title(m[2])

        loss = "Final train loss: {lo:e}"
        loss = loss.format(lo=list(hist['loss'])[-1])

        lrate = "Learning rate: {ep:.4f}"
        lrate = lrate.format(ep=m[3])

        plt.text(0.1, 0.55, loss,
                 ha='left', va='top',
                 transform=ax.transAxes,
                 color='blue'
                )
        plt.text(0.1, 0.45, lrate,
                 ha='left', va='top',
                 transform=ax.transAxes,
                 color='black'
                )


    plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 

    plt.show()


# CNN Model


In [9]:
def build_cnn_model(num_features,
                    hidden_layer_sizes=[],
                    activation='relu',
                    optimizer='SGD',
                    learning_rate=0.01):
    """
    Args:
    n_classes: Number of output classes in the dataset.
    hidden_layer_sizes: A list with the number of units in each hidden layer.
    activation: The activation function to use for the hidden layers.
    optimizer: The optimizer to use (SGD, Adam).
    learning_rate: The learning rate for the optimizer.

    Returns:
    model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)

    # YOUR CODE HERE
    model = tf.keras.Sequential()
    
    # Set input shape in flatteing layer
    model.add(tf.keras.layers.Input(shape=(num_features,), name='Input'))
    
    model.add(tf.keras.layers.Dense(
        units=43,                     # output dim
        input_shape=[num_features],  # input dim
        use_bias=True,               # use a bias (intercept) param
        kernel_initializer=tf.ones_initializer,  # initialize params to 1
        bias_initializer=tf.ones_initializer,    # initialize bias to 1
    ))
    
#     model.add(tf.keras.layers.Flatten(input_shape=(28, 28, 1), name='Flatten'))
    
    for hidden_layer_size in hidden_layer_sizes:
        model.add(tf.keras.layers.Dense(units=hidden_layer_size,
                                        activation=activation,
                                        name='Hidden_' + str(hidden_layer_size)))
#         model.add(BatchNormalization())
        
    model.add(tf.keras.layers.Dropout(rate=0.1, name='Dropout'))
    
    model.add(tf.keras.layers.Dense(units=43, activation='softmax', name='Output'))
    
#     opt = keras.optimizers.Adam(learning_rate=learning_rate)
#     opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.optimizer.learning_rate = learning_rate

    return model

In [12]:
NUM_BATCH = 32
EPSILON = 0.01
EPOCHS = 1000
SPLIT = 0.10

models = []

# for col in columns[1:]:
for col in ['HOG_Features']:
    train_features, labels = features[col], features['ClassId']
    split = int(train_features.shape[0] * SPLIT)
    val_features, v_labels = train_features[-split:], labels[-split:]
    train_features, t_labels = train_features[:-split], labels[:-split]
   
    tdataset = tf.data.Dataset.from_tensors(((np.vstack(train_features)), t_labels))
    vdataset = tf.data.Dataset.from_tensors(((np.vstack(val_features)), v_labels))
    
    tdataset.batch(NUM_BATCH)
    
    learningRate = EPSILON
    epochs = EPOCHS
#     learningRate = 0.1 if col == 'HOG_Features' else EPSILON
#     learningRate = 0.01 if col == 'LBP_Hist' else learningRate
#     epochs = 10 if col == 'LBP_Hist' else EPOCHS
    
    print(f'Model for {col}')
    print('='*100)
    num_features = train_features[0].len()
    model = build_cnn_model(num_features, [], activation='relu',
                            optimizer='Adam', learning_rate=learningRate)
   
    history = model.fit(tdataset, epochs=epochs, verbose=1, validation_data=vdataset)
    models.append((model, history, col, learningRate))
    print('='*100, '\n\n')


Model for HOG_Features
Epoch 1/1000


2023-12-01 23:32:36.884420: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [7740]
	 [[{{node Placeholder/_1}}]]


1/1 [==============================] - 1s 580ms/step - loss: 809.7340 - accuracy: 0.0260 - val_loss: 569.8172 - val_accuracy: 0.0000e+00
Epoch 2/1000
1/1 [==============================] - 0s 52ms/step - loss: 610.2277 - accuracy: 0.0264 - val_loss: 507.7613 - val_accuracy: 0.0000e+00
Epoch 3/1000
1/1 [==============================] - ETA: 0s - loss: 465.7734 - accuracy: 0.0252

2023-12-01 23:32:37.380733: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [860]
	 [[{{node Placeholder/_1}}]]


1/1 [==============================] - 0s 52ms/step - loss: 465.7734 - accuracy: 0.0252 - val_loss: 551.1492 - val_accuracy: 0.0000e+00
Epoch 4/1000
1/1 [==============================] - 0s 59ms/step - loss: 402.0034 - accuracy: 0.0255 - val_loss: 582.9549 - val_accuracy: 0.0000e+00
Epoch 5/1000
1/1 [==============================] - 0s 55ms/step - loss: 365.8684 - accuracy: 0.0270 - val_loss: 591.2582 - val_accuracy: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 48ms/step - loss: 326.4897 - accuracy: 0.0255 - val_loss: 610.3411 - val_accuracy: 0.0000e+00
Epoch 7/1000
1/1 [==============================] - 0s 48ms/step - loss: 301.8261 - accuracy: 0.0278 - val_loss: 627.9737 - val_accuracy: 0.0000e+00
Epoch 8/1000
1/1 [==============================] - 0s 47ms/step - loss: 289.8407 - accuracy: 0.0287 - val_loss: 618.7516 - val_accuracy: 0.0000e+00
Epoch 9/1000
1/1 [==============================] - 0s 49ms/step - loss: 266.0528 - accuracy: 0.0280 - val_loss: 680.30

...

1/1 [==============================] - 0s 78ms/step - loss: 0.4187 - accuracy: 0.8867 - val_loss: 240.6748 - val_accuracy: 0.0674
Epoch 730/1000
1/1 [==============================] - 0s 58ms/step - loss: 0.4096 - accuracy: 0.8828 - val_loss: 240.5679 - val_accuracy: 0.0674
Epoch 731/1000
1/1 [==============================] - 0s 53ms/step - loss: 0.4200 - accuracy: 0.8818 - val_loss: 240.4908 - val_accuracy: 0.0651
Epoch 732/1000
1/1 [==============================] - 0s 59ms/step - loss: 0.4273 - accuracy: 0.8907 - val_loss: 240.4068 - val_accuracy: 0.0651
Epoch 733/1000
1/1 [==============================] - 0s 54ms/step - loss: 0.4186 - accuracy: 0.8885 - val_loss: 240.3139 - val_accuracy: 0.0651
Epoch 734/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.3966 - accuracy: 0.8953 - val_loss: 240.2274 - val_accuracy: 0.0651
Epoch 735/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.4141 - accuracy: 0.8929 - val_loss: 240.1514 - val_accuracy: 0.0663


1/1 [==============================] - 0s 63ms/step - loss: 0.3620 - accuracy: 0.9059 - val_loss: 235.5314 - val_accuracy: 0.0651
Epoch 786/1000
1/1 [==============================] - 0s 59ms/step - loss: 0.3654 - accuracy: 0.9039 - val_loss: 235.4564 - val_accuracy: 0.0640
Epoch 787/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.3630 - accuracy: 0.9019 - val_loss: 235.3985 - val_accuracy: 0.0640
Epoch 788/1000
1/1 [==============================] - 0s 59ms/step - loss: 0.3565 - accuracy: 0.9041 - val_loss: 235.3326 - val_accuracy: 0.0640
Epoch 789/1000
1/1 [==============================] - 0s 59ms/step - loss: 0.3551 - accuracy: 0.9049 - val_loss: 235.2858 - val_accuracy: 0.0651
Epoch 790/1000
1/1 [==============================] - 0s 57ms/step - loss: 0.3489 - accuracy: 0.9071 - val_loss: 235.2279 - val_accuracy: 0.0663
Epoch 791/1000
1/1 [==============================] - 0s 60ms/step - loss: 0.3561 - accuracy: 0.9022 - val_loss: 235.1729 - val_accuracy: 0.0686


1/1 [==============================] - 0s 53ms/step - loss: 0.3185 - accuracy: 0.9161 - val_loss: 231.2678 - val_accuracy: 0.0674
Epoch 842/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.3118 - accuracy: 0.9196 - val_loss: 231.2535 - val_accuracy: 0.0663
Epoch 843/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.3128 - accuracy: 0.9186 - val_loss: 231.2385 - val_accuracy: 0.0651
Epoch 844/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.3070 - accuracy: 0.9202 - val_loss: 231.2161 - val_accuracy: 0.0663
Epoch 845/1000
1/1 [==============================] - 0s 53ms/step - loss: 0.3147 - accuracy: 0.9141 - val_loss: 231.1915 - val_accuracy: 0.0663
Epoch 846/1000
1/1 [==============================] - 0s 54ms/step - loss: 0.3151 - accuracy: 0.9185 - val_loss: 231.1270 - val_accuracy: 0.0663
Epoch 847/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.3211 - accuracy: 0.9163 - val_loss: 231.0540 - val_accuracy: 0.0663


1/1 [==============================] - 0s 61ms/step - loss: 0.2727 - accuracy: 0.9278 - val_loss: 227.3308 - val_accuracy: 0.0640
Epoch 898/1000
1/1 [==============================] - 0s 60ms/step - loss: 0.2746 - accuracy: 0.9275 - val_loss: 227.2385 - val_accuracy: 0.0640
Epoch 899/1000
1/1 [==============================] - 0s 54ms/step - loss: 0.2764 - accuracy: 0.9245 - val_loss: 227.1624 - val_accuracy: 0.0640
Epoch 900/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.2774 - accuracy: 0.9260 - val_loss: 227.0984 - val_accuracy: 0.0651
Epoch 901/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.2718 - accuracy: 0.9280 - val_loss: 227.0153 - val_accuracy: 0.0663
Epoch 902/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.2721 - accuracy: 0.9264 - val_loss: 226.9524 - val_accuracy: 0.0674
Epoch 903/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.2766 - accuracy: 0.9278 - val_loss: 226.8994 - val_accuracy: 0.0686


1/1 [==============================] - 0s 58ms/step - loss: 0.2472 - accuracy: 0.9358 - val_loss: 224.5973 - val_accuracy: 0.0628
Epoch 954/1000
1/1 [==============================] - 0s 55ms/step - loss: 0.2522 - accuracy: 0.9339 - val_loss: 224.5580 - val_accuracy: 0.0628
Epoch 955/1000
1/1 [==============================] - 0s 54ms/step - loss: 0.2502 - accuracy: 0.9322 - val_loss: 224.4970 - val_accuracy: 0.0651
Epoch 956/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.2455 - accuracy: 0.9368 - val_loss: 224.4418 - val_accuracy: 0.0651
Epoch 957/1000
1/1 [==============================] - 0s 50ms/step - loss: 0.2458 - accuracy: 0.9327 - val_loss: 224.4127 - val_accuracy: 0.0651
Epoch 958/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.2450 - accuracy: 0.9351 - val_loss: 224.3850 - val_accuracy: 0.0651
Epoch 959/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.2426 - accuracy: 0.9354 - val_loss: 224.3766 - val_accuracy: 0.0651


In [ ]:
# def evaluate_model(num_features,
#                    hidden_layer_sizes=[],
#                    activation='tanh',
#                    optimizer='Adam',
#                    learning_rate=0.01,
#                    num_epochs=5):

#     # Build the model.
#     model = build_model(num_features=num_features,
#                         hidden_layer_sizes=hidden_layer_sizes,
#                         activation=activation,
#                         optimizer=optimizer,
#                         learning_rate=learning_rate)

#     # Train the model.
#     print('Training...')
#     history = model.fit(x=X_train,
#                         y=Y_train,
#                         epochs=num_epochs,
#                         batch_size=64,
#                         validation_split=0.1,
#                         verbose=0)

#     # Retrieve the training metrics (after each train epoch) and the final test
#     # accuracy.
#     train_accuracy = history.history['accuracy']
#     val_accuracy = history.history['val_accuracy']
#     plt.plot(train_accuracy, label='train_accuracy')
#     plt.plot(val_accuracy, label='validation accuracy')
#     plt.xticks(range(num_epochs))
#     plt.xlabel('Train epochs')
#     plt.legend()
#     plt.show()
    
#     test_accuracy = model.evaluate(x=X_test, y=Y_test, verbose=0,
#                                    return_dict=True)['accuracy']
#     return {'accuracy':test_accuracy, 'model':model}

# print('Test Accuracy: %1.4f' %train_and_evaluate()['accuracy'])